# 01-OpenAI的API调用

! pip install openai langchain-openai

In [2]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI


load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_API_BASE")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", api_key=api_key, base_url=base_url, temperature=0.3)

llm.invoke("中国的首都是哪里?")


AIMessage(content='中国的首都是北京。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 16, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-63b2cd27-8e9e-488e-992f-2a1bb5231f6d-0', usage_metadata={'input_tokens': 16, 'output_tokens': 7, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 基本示例: 提示+模型+输出解析器

最基本和常见的用例是将提示模板和模型链接在一起。为了看看这是如何工作的，让我们创建一个接受主题并生成小红书短文的链：

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("请帮我生成一篇关于{topic}的小红书短文")
model = ChatOpenAI(model_name="gpt-3.5-turbo", api_key=api_key, base_url=base_url, temperature=0.3)

chain = prompt | model | StrOutputParser()

chain.invoke({"topic": "胖东来烤鸭"})


'标题：胖东来烤鸭，让味蕾沉醉的美味\n\n在繁华的都市中，有一家让人回味无穷的烤鸭店——胖东来烤鸭。这家店位于市中心，每天都吸引着无数食客前来品尝他们独特的烤鸭。\n\n走进店内，一股浓浓的烤鸭香味扑面而来，让人垂涎欲滴。烤鸭师傅们正在烤制鸭子，火候恰到好处，皮脆肉嫩，色泽金黄诱人。每一只烤鸭都经过精心处理，保证了口感的鲜美。\n\n品尝胖东来烤鸭，首先让人感受到的是鸭皮的香脆和肉质的鲜嫩，入口即化。配上店家特制的酱料和蔬菜，更是美味无比。不仅如此，店内还提供各种配菜和小吃，让人吃得过瘾。\n\n除了烤鸭，胖东来还有各种口味的炒饭、炒面等美食，满足了不同人的口味需求。无论是单人用餐还是家庭聚餐，这里都是一个不错的选择。\n\n胖东来烤鸭，让人们在繁忙的生活中找到了一处放松的天堂。每一次品尝都是一次美味的享受，让人的味蕾沉醉其中。如果你还没有来过胖东来，赶紧来一趟吧，让你的味蕾也沉醉在这份美味之中吧！'

## Prompt提示

prompt 是一个 BasePromptTemplate ，这意味着它接受模板变量的字典并生成一个 PromptValue 。 PromptValue 是一个完整提示的包装器，可以传递给 LLM （它接受一个字符串作为输入）或 ChatModel （它接受一个序列作为输入的消息）。它可以与任何一种语言模型类型一起使用，因为它定义了生成 BaseMessage 和生成字符串的逻辑。

In [4]:
prompt = ChatPromptTemplate.from_template("请根据下面的主题，帮我生成一篇小红书营销短文: {topic}")
prompt_value = prompt.invoke({"topic": "康师傅绿茶"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='请根据下面的主题，帮我生成一篇小红书营销短文: 康师傅绿茶', additional_kwargs={}, response_metadata={})])

## Model模型

Prompt_value被传给LLM，在本例中我们的LLM是ChatModel，意味着它将输BaseMessage

In [5]:
message = model.invoke(prompt_value)
message

AIMessage(content='标题：探寻康师傅绿茶的美味之道\n\n大家好，今天我想和大家分享的是我最近发现的一款超级好喝的饮品——康师傅绿茶。作为一个绿茶爱好者，我一直在寻找一款口感清爽、回甘适中的绿茶，而康师傅绿茶完美地符合了我的要求。\n\n首先，康师傅绿茶选用的是优质绿茶叶，经过精心炒制而成，保留了绿茶的原汁原味，清香怡人。喝下去，一股清爽的茶香扑面而来，仿佛置身于茶园之中，让人心情愉悦。\n\n其次，康师傅绿茶口感醇厚，入口滑润，回甘十足。喝完一口，满口生津，仿佛舌尖上跳动着清泉，让人回味无穷。而且，康师傅绿茶不添加防腐剂和人工色素，健康又放心。\n\n最重要的是，康师傅绿茶不仅口感好，包装设计也非常时尚。瓶身简约大方，绿色的包装让人一眼就能认出，是一款高颜值的饮品。无论是携带出门还是在家享用，都能为生活增添一份美好。\n\n总的来说，康师傅绿茶是一款值得推荐的好饮品，清香回甘，口感醇厚，健康放心。无论是工作间隙还是休闲时光，都可以随时随地享用，让生活更加美好。快来试试康师傅绿茶，探寻它的美味之道吧！#康师傅绿茶# #清爽回甘# #健康生活#', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 612, 'prompt_tokens': 47, 'total_tokens': 659, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d86556c4-a661-4523-8cf4-c277ab757907-0', usage_m

## Output Parser输出解析器

将 model 输出传递给 output_parser ，这是一个 BaseOutputParser ，意味着它接受字符串或 BaseMessage 作为输入。 StrOutputParser 特别简单地将任何输入转换为字符串。

input ->(dict) -> PromptTemplate -> (prompt_value) -> ChatModel -> (ChatMessage) -> OutputParser -> (str) -> result

In [6]:
response = StrOutputParser().invoke(message)
response


'标题：探寻康师傅绿茶的美味之道\n\n大家好，今天我想和大家分享的是我最近发现的一款超级好喝的饮品——康师傅绿茶。作为一个绿茶爱好者，我一直在寻找一款口感清爽、回甘适中的绿茶，而康师傅绿茶完美地符合了我的要求。\n\n首先，康师傅绿茶选用的是优质绿茶叶，经过精心炒制而成，保留了绿茶的原汁原味，清香怡人。喝下去，一股清爽的茶香扑面而来，仿佛置身于茶园之中，让人心情愉悦。\n\n其次，康师傅绿茶口感醇厚，入口滑润，回甘十足。喝完一口，满口生津，仿佛舌尖上跳动着清泉，让人回味无穷。而且，康师傅绿茶不添加防腐剂和人工色素，健康又放心。\n\n最重要的是，康师傅绿茶不仅口感好，包装设计也非常时尚。瓶身简约大方，绿色的包装让人一眼就能认出，是一款高颜值的饮品。无论是携带出门还是在家享用，都能为生活增添一份美好。\n\n总的来说，康师傅绿茶是一款值得推荐的好饮品，清香回甘，口感醇厚，健康放心。无论是工作间隙还是休闲时光，都可以随时随地享用，让生活更加美好。快来试试康师傅绿茶，探寻它的美味之道吧！#康师傅绿茶# #清爽回甘# #健康生活#'

## 流式输出

如果我们想流式传输结果，我们需要更改我们的函数，使其返回一个生成器。通过for循环打印输出效果

In [7]:

chain = prompt | model | StrOutputParser()

for chunk in chain.stream({"topic": "康师傅绿茶"}):
    print(chunk, end="", flush=True)


标题：探寻康师傅绿茶的清新之味

大家好，今天我要和大家分享的是康师傅绿茶，一款清新怡人的饮品。康师傅绿茶选用优质绿茶叶，经过精心炒制而成，保留了绿茶的原汁原味，让人一口喝下去就仿佛置身于清新的茶园中。

康师傅绿茶不仅口感清爽，而且富含丰富的维生素和矿物质，对身体健康有益。无论是在炎炎夏日还是在寒冷冬季，一瓶康师傅绿茶都能为你带来清凉和舒适。

除了口感和营养价值，康师傅绿茶还有着独特的包装设计，简约大方，让人一眼就能被吸引。无论是在办公室、户外运动还是在家休闲，都能随时享受到康师傅绿茶带来的清新之味。

总的来说，康师傅绿茶是一款不可多得的好饮品，无论是口感、营养还是包装设计都让人满意。如果你也喜欢清新怡人的味道，不妨试试康师傅绿茶，让清新之味伴随你的每一天！